In [17]:
import pandas as pd

In [18]:
catalog.list()

['p18_19_0', 'p18_19_1', 'p18_19_3', 'p18_19_5', 'qld_p18_19', 'parameters']

In [19]:
df = catalog.load('p18_19_5')

2023-04-24 22:02:29,071 - kedro.io.data_catalog - INFO - Loading data from 'p18_19_5' (CSVDataSet)...


In [20]:
df.head()

,0,1,2,3,4,5
0,NaN,Headcount,NaN,NaN,Percentage,NaN
1,Other entities,Female,Male,Total,Female,Male
2,Legal Aid Queensland,480,143,623,0.7704654895666132,0.22953451043338685
3,Office of the Health Ombudsman,100,42,142,0.704225352112676,0.29577464788732394
4,Queensland Art Gallery,234,131,365,0.6410958904109589,0.3589041095890411


In [21]:
# Create a boolean is_title_mask indicating which rows have only one non-null entry
is_title_mask = df.count(axis=1) == 1

In [22]:
table_name = "None"

# Check if the dataframe contains a row with only one non-null value. The non-value is assumed to become the title of the table
if is_title_mask.any():

    # Find the column index of the non-null value in each row by filtering the DataFrame to include only the rows with one non-null entry
    row_index = df[is_title_mask].index.tolist()[0]

    # Find the column index of the non-null value in the row
    col_index = df[is_title_mask].notnull().idxmax(axis=1)

    # Extract the value of the cell as a string for naming the table
    table_name = df.loc[row_index, col_index].item()

    # Drop  the first row in the dataframe
    df = df.drop(index=0)

    # Reset the dataframe's row index
    df = df.reset_index(drop=True)
else:
    print("No title")

No title


In [23]:
print(f"table name: {table_name}")

table name: None


In [24]:
df.head()

,0,1,2,3,4,5
0,NaN,Headcount,NaN,NaN,Percentage,NaN
1,Other entities,Female,Male,Total,Female,Male
2,Legal Aid Queensland,480,143,623,0.7704654895666132,0.22953451043338685
3,Office of the Health Ombudsman,100,42,142,0.704225352112676,0.29577464788732394
4,Queensland Art Gallery,234,131,365,0.6410958904109589,0.3589041095890411


In [25]:
# The row index of the last row with null values in the dataframe
row_index_last_null = df.isna().iloc[::-1].idxmax()[0]

print(f"The index of the last row with null :{row_index_last_null}")

# If the only first row has null values, then assume that the second row has column index values
if row_index_last_null == 0:
    row_index_last_null =+ 1

The index of the last row with null :0


In [26]:
# For first rows in the dataframe with null values, populate the null values with the preceding non-null value
df.iloc[:row_index_last_null] = df.iloc[:row_index_last_null].fillna(method='ffill', axis=1)

In [27]:
df.head()

,0,1,2,3,4,5
0,NaN,Headcount,Headcount,Headcount,Percentage,Percentage
1,Other entities,Female,Male,Total,Female,Male
2,Legal Aid Queensland,480,143,623,0.7704654895666132,0.22953451043338685
3,Office of the Health Ombudsman,100,42,142,0.704225352112676,0.29577464788732394
4,Queensland Art Gallery,234,131,365,0.6410958904109589,0.3589041095890411


In [28]:
# Define a function to concatenate the non-null values
def concat_non_null(x):
    return '_'.join([str(val) for val in x.values if not pd.isnull(val)])

# Merge the rows of the dataframe by concatenating the non-null values
merged_row = df.iloc[:2].apply(concat_non_null).to_frame().T

print(f"Merged row: {merged_row}")

Merged row:                 0                 1               2                3  \
0  Other entities  Headcount_Female  Headcount_Male  Headcount_Total   

                   4                5  
0  Percentage_Female  Percentage_Male  


In [29]:
# Drop the rows up to row_index_last_null in the dataframe
df = df.drop(index=range(0,row_index_last_null+1))

# Reset the dataframe's row index
df = df.reset_index(drop=True)

In [30]:
df.head()

,0,1,2,3,4,5
0,Legal Aid Queensland,480,143,623,0.7704654895666132,0.22953451043338685
1,Office of the Health Ombudsman,100,42,142,0.704225352112676,0.29577464788732394
2,Queensland Art Gallery,234,131,365,0.6410958904109589,0.3589041095890411
3,Queensland Family and Child Commission,56,17,73,0.7671232876712328,0.2328767123287671
4,Queensland Human Rights Commission,35,9,44,0.7954545454545454,0.20454545454545456


In [31]:
# Set the merged_row as the dataframe's column index
df.columns = merged_row.iloc[0]

In [32]:
df

,Other entities,Headcount_Female,Headcount_Male,Headcount_Total,Percentage_Female,Percentage_Male
0,Legal Aid Queensland,480,143,623,0.7704654895666132,0.22953451043338685
1,Office of the Health Ombudsman,100,42,142,0.704225352112676,0.29577464788732394
2,Queensland Art Gallery,234,131,365,0.6410958904109589,0.3589041095890411
3,Queensland Family and Child Commission,56,17,73,0.7671232876712328,0.2328767123287671
4,Queensland Human Rights Commission,35,9,44,0.7954545454545454,0.20454545454545456
5,Queensland Museum,219,110,329,0.6656534954407295,0.3343465045592705
6,Resources Safety and Health Queensland,120,176,296,0.40540540540540543,0.5945945945945946
7,State Library of Queensland,255,98,353,0.7223796033994334,0.2776203966005666
8,Trade and Investment Queensland,83,52,135,0.6148148148148148,0.3851851851851852
9,Sector sub-total: Other entities,1582,778,2360,0.6703389830508475,0.32966101694915256
